In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import sqlite3

In [2]:
pd.set_option('display.max_columns', 50)

In [3]:
nppes_cols = ['NPI', 
              'Entity Type Code', 
              'Provider Organization Name (Legal Business Name)',
              'Provider Name Prefix Text',
              'Provider First Name',
              'Provider Middle Name',
              'Provider Last Name (Legal Name)', 
              'Provider Name Suffix Text',
              'Provider Credential Text',
              'Provider First Line Business Practice Location Address',
              'Provider Second Line Business Practice Location Address',
              'Provider Business Practice Location Address City Name',
              'Provider Business Practice Location Address State Name',
              'Provider Business Practice Location Address Postal Code',
              'Healthcare Provider Taxonomy Code_1',
              'Healthcare Provider Primary Taxonomy Switch_1',
              'Healthcare Provider Taxonomy Code_2',
              'Healthcare Provider Primary Taxonomy Switch_2',
              'Healthcare Provider Taxonomy Code_3',
              'Healthcare Provider Primary Taxonomy Switch_3',
              'Healthcare Provider Taxonomy Code_4',
              'Healthcare Provider Primary Taxonomy Switch_4',
              'Healthcare Provider Taxonomy Code_5',
              'Healthcare Provider Primary Taxonomy Switch_5',
              'Healthcare Provider Taxonomy Code_6',
              'Healthcare Provider Primary Taxonomy Switch_6',
              'Healthcare Provider Taxonomy Code_7',
              'Healthcare Provider Primary Taxonomy Switch_7',
              'Healthcare Provider Taxonomy Code_8',
              'Healthcare Provider Primary Taxonomy Switch_8',
              'Healthcare Provider Taxonomy Code_9',
              'Healthcare Provider Primary Taxonomy Switch_9',
              'Healthcare Provider Taxonomy Code_10',
              'Healthcare Provider Primary Taxonomy Switch_10',
              'Healthcare Provider Taxonomy Code_11',
              'Healthcare Provider Primary Taxonomy Switch_11',
              'Healthcare Provider Taxonomy Code_12',
              'Healthcare Provider Primary Taxonomy Switch_12',
              'Healthcare Provider Taxonomy Code_13',
              'Healthcare Provider Primary Taxonomy Switch_13',
              'Healthcare Provider Taxonomy Code_14',
              'Healthcare Provider Primary Taxonomy Switch_14',
              'Healthcare Provider Taxonomy Code_15',
              'Healthcare Provider Primary Taxonomy Switch_15']


nppes = pd.read_csv('../data/NPPES_Data_Dissemination_February_2022/npidata_pfile_20050523-20220213.csv',
                    usecols = nppes_cols, 
                    dtype = str, 
                    chunksize = 100000, 
                    low_memory = False)


tn_providers = pd.DataFrame()


for chunk in tqdm(nppes):
    filtered_chunk = chunk[nppes_cols].query('`Provider Business Practice Location Address State Name` == "TN"')
    tn_providers = tn_providers.append(filtered_chunk)


tn_providers.columns = ['npi', 
                        'entity_type_code', 
                        'organization_name',
                        'name_prefix', 
                        'first_name', 
                        'middle_name',
                        'last_name',
                        'name_suffix',
                        'credentials', 
                        'address_first_line', 
                        'address_second_line', 
                        'city',
                        'state', 
                        'zipcode',
                        'taxonomy_code_1',
                        'primary_taxonomy_switch_1',
                        'taxonomy_code_2',
                        'primary_taxonomy_switch_2',
                         'taxonomy_code_3',
                        'primary_taxonomy_switch_3',
                         'taxonomy_code_4',
                        'primary_taxonomy_switch_4',
                         'taxonomy_code_5',
                        'primary_taxonomy_switch_5',
                         'taxonomy_code_6',
                        'primary_taxonomy_switch_6',
                         'taxonomy_code_7',
                        'primary_taxonomy_switch_7',
                         'taxonomy_code_8',
                        'primary_taxonomy_switch_8',
                         'taxonomy_code_9',
                        'primary_taxonomy_switch_9',
                         'taxonomy_code_10',
                        'primary_taxonomy_switch_10',
                         'taxonomy_code_11',
                        'primary_taxonomy_switch_11',
                         'taxonomy_code_12',
                        'primary_taxonomy_switch_12',
                         'taxonomy_code_13',
                        'primary_taxonomy_switch_13',
                         'taxonomy_code_14',
                        'primary_taxonomy_switch_14',
                         'taxonomy_code_15',
                        'primary_taxonomy_switch_15']

0it [00:00, ?it/s]

In [4]:
tn_providers['zipcode'] = tn_providers['zipcode'].astype(str).str[:5]


cbsa = pd.read_excel('../data/ZIP_CBSA_122021.xlsx', dtype = {'zip': str, 'cbsa': str}) \
    .query('usps_zip_pref_state == "TN"') \
    .sort_values('tot_ratio', ascending = False) \
    .drop_duplicates(subset = 'zip') \
    [['zip', 'cbsa', 'usps_zip_pref_city', 'usps_zip_pref_state']]


cbsa.columns = ['zipcode', 'cbsa', 'cbsa_city', 'cbsa_state']


tn_providers = tn_providers.merge(cbsa, on = 'zipcode')

In [6]:
id_cols = ['npi', 
           'entity_type_code', 
           'organization_name',
           'name_prefix', 
           'first_name', 
           'middle_name',
           'last_name',
           'name_suffix',
           'credentials', 
           'address_first_line', 
           'address_second_line', 
           'city',
           'state', 
           'zipcode',
           'cbsa', 
           'cbsa_city', 
           'cbsa_state']


var_cols = ['taxonomy_code', 
            'primary_taxonomy_switch']


tn_providers = pd.wide_to_long(tn_providers, stubnames = var_cols, i = id_cols, j = '', sep = '_') \
    .sort_values('npi') \
    .query('primary_taxonomy_switch == "Y"') \
    .reset_index() \
    .drop(['', 'primary_taxonomy_switch'], axis = 1)

In [7]:
nucc = pd.read_csv('../data/nucc_taxonomy_220.csv').drop(['Definition', 'Notes'], axis = 1)


nucc.columns = ['taxonomy_code',
                'taxonomy_grouping',
                'taxonomy_classification',
                'taxonomy_specialization',
                'taxonomy_display_name',
                'taxonomy_section']


tn_providers = tn_providers.merge(nucc, on = 'taxonomy_code')

In [11]:
tn_providers.to_csv('../data/tn_providers.csv', index = False)

In [9]:
hop_teaming = pd.read_csv('../data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv',
                          dtype = {'from_npi': str, 'to_npi': str},
                          chunksize = 1000000,
                          low_memory = False)


entity_type = dict(tn_providers[['npi', 'entity_type_code']].values)


tn_hop_teaming = pd.DataFrame()


for chunk in tqdm(hop_teaming):
    filtered_chunk = chunk[(chunk['from_npi'].map(entity_type) == '1') & 
                           (chunk['to_npi'].map(entity_type) == '2') &
                           (chunk['transaction_count'] >= 50) &
                           (chunk['average_day_wait'] < 50)]
    tn_hop_teaming = tn_hop_teaming.append(filtered_chunk)

0it [00:00, ?it/s]

In [12]:
tn_hop_teaming.to_csv('../data/tn_hop_teaming.csv', index = False)